In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import re
import string
from collections import Counter
from typing import List
import multiprocessing as mp
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
qasar_t_dir = "/content/drive/My Drive/quasar/finetunebertsquad"

In [14]:
import os
print(os.listdir(qasar_t_dir))

['finetune_squad_1_1.ipynb', 'uncased_L-12_H-768_A-12.zip', 'model_repo', 'modeling.py', 'optimization.py', 'run_squad.py', 'tokenization.py', 'dev-v1.1.json', 'evaluate-v1.1.py', 'train-v1.1.json']


### Create directory where the pre trained embedding will be downloaded

In [15]:
%cd /content/drive/My Drive/quasar/finetunebertsquad

/content/drive/My Drive/quasar/finetunebertsquad


In [16]:
!pwd

/content/drive/My Drive/quasar/finetunebertsquad


### download the relavent files and model from official google storage location.This will be done only once. later the code can be commented out

In [0]:
model_download = False #flag this to True when you are downloading the model first time

In [10]:
if model_download:
  !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-05-06 15:12:44--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  83.1MB/s    in 4.7s    

2020-05-06 15:12:48 (83.1 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [0]:
import zipfile
if model_download:
  repo = 'model_repo'
  with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
      zip_ref.extractall(repo)

In [12]:
!ls 'model_repo/uncased_L-12_H-768_A-12'

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [13]:
if model_download:
  !wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
  !wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
  !wget https://raw.githubusercontent.com/google-research/bert/master/run_squad.py 
  !wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

--2020-05-06 15:13:23--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.01s   

2020-05-06 15:13:23 (3.79 MB/s) - ‘modeling.py’ saved [37922/37922]

--2020-05-06 15:13:25--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6258 (6.1K) [text/plain]
Saving to: ‘optimizati

In [0]:
if model_download:
  !mkdir ./model_repo/output #save the fine tuned model

In [18]:
import tensorflow
tensorflow.__version__

'1.15.2'

### Tensor flow should be less than 2.0. Please un comments these lines and un unstall version 2.0 or higher

In [16]:
#!pip uninstall tensorflow -y
#!pip install tensorflow==1.15.2

Uninstalling tensorflow-2.2.0rc4:
  Successfully uninstalled tensorflow-2.2.0rc4
     |████████████████████████████████| 110.5MB 39kB/s 
     |████████████████████████████████| 512kB 55.0MB/s 
     |████████████████████████████████| 3.8MB 52.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=00a3b1a486897ba8eae2bc2fadd033e0856876f99c8b353c75f3ee84d27ead62
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2

In [9]:
import tensorflow
tensorflow.__version__

'1.15.2'

### Run the finetuning module on Quasar train and predict on quasar dev. Save the training model and predictions inside ./model_repo/output/
#### Now go to model_repo/uncased_L-12_H-768_A-12 directory and open bert_config.json and manually change the model to new finetuned model. Some more details can be found at "https://github.com/google-research/bert#squad-11"

In [10]:
!pwd

/content/drive/My Drive/quasar/finetunebertsquad


In [0]:
!python run_squad.py \
  --vocab_file=./model_repo/uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=./model_repo/uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=./model_repo/uncased_L-12_H-768_A-12/bert_model.ckpt \
  --do_train=True \
  --train_file=train-v1.1.json \
  --do_predict=True \
  --predict_file=../quasar_to_squad/quasar-t_to_squad_long_dev_out.json \
  --train_batch_size=12 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=./model_repo/output/




W0506 16:06:54.516788 140253393602432 module_wrapper.py:139] From run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0506 16:06:54.516972 140253393602432 module_wrapper.py:139] From run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0506 16:06:54.517105 140253393602432 module_wrapper.py:139] From /content/drive/My Drive/quasar/finetunebertsquad/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0506 16:06:54.518916 140253393602432 module_wrapper.py:139] From run_squad.py:1133: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.co

#### Predictions - though we will only use the finetuned model and add to the config file and later predictions will be made using custom functions because they give the confidence score and start position of the predctions along with EM and F1 scores.

In [0]:
#python evaluate-v1.1.py quasar-t_to_squad_dev_questions_out.json ./squad/predictions.json